<a href="https://colab.research.google.com/github/JJChrzanowski/GlyCulator3_hotfix/blob/main/DayByDay_GlyCulator_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day-by-day GlyCulator API analysis

Please follow these steps:

1. Enter your original_session_id and api_key in the variables below.
2. Run this cell.

The code will:
 - Connect to the GlyCulator API
 - Retrieve the original analysis details
 - Perform day-by-day analysis on each file and date range
 - Show you the final results as a table
 - Provide a link to download the results as a CSV file

In [ ]:
# @title
# Run once to import required functions
import requests
import datetime
import time
import pandas as pd

def get_analysis_details(session_id, api_key, base_url="https://glyculator.btm.umed.pl"):
    """Retrieve details about an existing analysis."""
    url = f"{base_url}/api/analyses/{session_id}?api_key={api_key}"
    response = requests.get(url, headers={"accept": "application/json"})
    response.raise_for_status()
    return response.json()

def create_analysis(api_key, data, base_url="https://glyculator.btm.umed.pl"):
    """Create a new analysis using the provided JSON data."""
    url = f"{base_url}/api/analyses/new?api_key={api_key}"
    headers = {"Content-Type": "application/json", "accept": "application/json"}
    response = requests.post(url, json=data, headers=headers)
    response.raise_for_status()
    return response.json()

def wait_for_analysis_finish(session_id, api_key, base_url="https://glyculator.btm.umed.pl", timeout=600, interval=10):
    """Polls the API until the analysis is finished or timeout is reached."""
    start_time = time.time()
    while time.time() - start_time < timeout:
        details = get_analysis_details(session_id, api_key, base_url=base_url)
        if details.get("finished"):
            return details
        time.sleep(interval)
    raise TimeoutError(f"Analysis {session_id} did not finish within {timeout} seconds.")

def generate_daily_ranges(start_date_str, end_date_str):
    """Generate a list of (date_from, date_to) tuples for each day in the range [start_date, end_date]."""
    start_date = datetime.date.fromisoformat(start_date_str)
    end_date = datetime.date.fromisoformat(end_date_str)

    # Assuming end_date is inclusive
    day_ranges = []
    current_date = start_date
    while current_date <= end_date:
        day_from = current_date
        day_to = current_date + datetime.timedelta(days=1)
        day_ranges.append((day_from.isoformat(), day_to.isoformat()))
        current_date += datetime.timedelta(days=1)
    return day_ranges

def flatten_indices(indices):
    """Flatten the nested indices dict into a single-level dict with period-specific prefixes."""
    flat = {}
    for period in ["whole", "day", "night"]:
        if period in indices:
            for k, v in indices[period].items():
                flat[f"{period}_{k}"] = v
    return flat

def remove_analysis_by_session(session_id, api_key, base_url="https://glyculator.btm.umed.pl"):
    """
    Cleanup function to remove an analysis by session_id.
    This function calls the GET endpoint:
    /api/remove_analysis_by_session?session_id={session_id}&api_key={api_key}
    """
    url = f"{base_url}/api/remove_analysis_by_session?session_id={session_id}&api_key={api_key}"
    response = requests.get(url, headers={"accept": "*/*"})
    response.raise_for_status()
    print(f'Session {session_id} removed as cleanup.')

def compute_day_by_day_indices(original_session_id, api_key, cleanup=False):
    # 1. Get the original analysis details
    original_details = get_analysis_details(original_session_id, api_key)

    # Extract the configuration parameters
    imputation_method = original_details.get("imputation_method")
    imputation_max_gap = original_details.get("imputation_max_gap")
    imputation_padding = original_details.get("imputation_padding")
    tbr_low = original_details.get("tbr_low")
    tbr_high = original_details.get("tbr_high")
    tar_low = original_details.get("tar_low")
    tar_high = original_details.get("tar_high")
    glucose_unit = original_details.get("glucose_unit")
    night_start = original_details.get("night_start")
    night_end = original_details.get("night_end")

    # The original name can be reused or modified
    base_name = original_details.get("name", "DayByDay_Analysis")

    # 2. Extract the analysis_files from the original
    analysis_files = original_details.get("analysis_files", [])

    # We'll store results here
    day_by_day_results = []

    # 3. For each analysis_file, generate daily segments and run new analyses
    for file_index, analysis_file in enumerate(analysis_files):
        file_hash = analysis_file["file"]["file_hash"]
        filename = analysis_file["file"]["filename"]

        date_from = analysis_file["date_from"]
        date_to = analysis_file["date_to"]

        # Generate daily ranges
        daily_ranges = generate_daily_ranges(date_from, date_to)

        for i, (day_start, day_end) in enumerate(daily_ranges):
            # Build request body for a new analysis
            new_analysis_data = {
                "name": f"{base_name}_File{file_index+1}_Day{i+1}",
                "imputation_method": imputation_method,
                "imputation_max_gap": imputation_max_gap,
                "imputation_padding": imputation_padding,
                "tbr_low": tbr_low,
                "tbr_high": tbr_high,
                "tar_low": tar_low,
                "tar_high": tar_high,
                "glucose_unit": glucose_unit,
                "night_start": night_start,
                "night_end": night_end,
                "analysis_files": [
                    {
                        "date_from": day_start,
                        "date_to": day_end,
                        "file": {
                            "file_hash": file_hash
                        }
                    }
                ]
            }

            # 4. Create a new analysis for this single-day segment
            try:
              creation_response = create_analysis(api_key, new_analysis_data)
              new_session_id = creation_response.get("session_id")

              if not new_session_id:
                  print("No session_id returned from new analysis creation. Skipping.")
                  continue

              # 5. Wait for the analysis to finish
              finished_details = wait_for_analysis_finish(new_session_id, api_key)

              # Extract day-by-day indices
              if "analysis_files" in finished_details and finished_details["analysis_files"]:
                  day_indices = finished_details["analysis_files"][0]["indices"]

                  # Append results with filename and file_hash included
                  day_by_day_results.append({
                      "session_id": new_session_id,
                      "file_hash": file_hash,
                      "filename": filename,
                      "date_from": day_start,
                      "date_to": day_end,
                      "indices": day_indices
                  })

                  # Cleanup a particular session once done if requested
                  if cleanup:
                      remove_analysis_by_session(new_session_id, api_key)

              # Print progress
              print(f"Completed day-by-day analysis for {day_start} to {day_end}, session_id: {new_session_id}")
            except:
              print(f"Error creating new analysis for {day_start} to {day_end} - probably no data for given day. Skipping.")

    # Convert day_by_day_results to a pandas DataFrame
    records = []
    for res in day_by_day_results:
        flat_indices = flatten_indices(res["indices"])
        record = {
            "session_id": res["session_id"],
            "file_hash": res["file_hash"],
            "filename": res["filename"],
            "date_from": res["date_from"],
            "date_to": res["date_to"]
        }
        # Merge flattened indices into the record
        record.update(flat_indices)
        records.append(record)

    df = pd.DataFrame(records)
    return df

In [ ]:
# NEVER SHARE YOUR API KEY!!!
original_session_id = "YOUR_SESSION_ID"
api_key = "YOUR_API_KEY"

In [ ]:
# Example use
df_results = compute_day_by_day_indices(original_session_id, api_key, cleanup=False)
df_results.head()